### Package

In [18]:
%%file fastica.py

import numpy as np
import pandas as pd
import scipy.linalg as la
import numba
from numba import jit

#whiten
def whiten2(X):
    X = X - X.mean(-1)[:, None]
    A = np.dot(X, X.T)
    D, P = la.eigh(A)
    D = np.diag(D)
    S_inv = np.sqrt(np.linalg.pinv(D))
    V = np.dot(S_inv, P.T)
    return np.dot(V, X), V
def whiten(X, n_components):
    X = X - X.mean(-1)[:, None]
    u, d, _ = la.svd(X, full_matrices=False)
    V = (u / d).T[:n_components]
    X *= np.sqrt(X.shape[1])
    
    return np.dot(V, X), V
#three functions
def _logcosh(x):
    gx = np.tanh(x)
    g_x = 1- gx ** 2
    return gx, g_x.mean(-1)

def _exp(x):
    exp = np.exp(-(x ** 2) / 2)
    gx = x * exp
    g_x = (1 - x ** 2) * exp
    return gx, g_x.mean(axis=-1)

def _cube(x):
    return x ** 3, (3 * x ** 2).mean(axis=-1)

def decorrelation(W):
    s, u = la.eigh(np.dot(W, W.T))
    return np.dot(np.dot(u * (1. / np.sqrt(s)), u.T), W)

# fastICA
def fastICA(X, fun='logcosh',n_components=None, maxIter = 200, tol = 1e-04):
    X = X.T
    n, m = X.shape
    if n_components is None:
        n_components = n
    X1,V = whiten(X,n_components)
    p = float(m)
    if fun == 'logcosh':
        g = _logcosh
    elif fun == 'exp':
        g = _exp
    elif fun == 'cube':
        g = _cube
    
    #initialize w with normal distribution
    W = np.asarray(np.random.normal(size=(n_components,n_components)))

    # calculate w iteratively
  
    for ii in range(maxIter):
        gwtx, g_wtx = g(np.dot(W, X1))
        W1 = decorrelation(np.dot(gwtx, X1.T) / p - g_wtx[:, None] * W)
        lim = max(abs(abs(np.diag(np.dot(W1, W.T))) - 1))
        W = W1
        if lim < tol:
            break
    S = np.dot(np.dot(W, V), X).T
    return V,W,S
if __name__ == '__main__':
    fastICA()

Overwriting fastica.py


In [1]:
%%file setup.py
from setuptools import setup
import setuptools

setup(name = "fastica",
      version = "1.0",
      author='Zhechang Yang and Xi Chen',
      author_email='zhechang.yang@duke.edu',
      url='http://people.duke.edu/~ccc14/sta-663-2018/',
      py_modules = ['fastica'],
      packages=setuptools.find_packages(),
      scripts = ['fastica.py'],
      python_requires='>=3',
      )

Overwriting setup.py


In [5]:
%%file README.md

fastica algorithm

Writing README.md


In [26]:
%%bash

python setup.py sdist

running sdist
running egg_info
writing fastica.egg-info/PKG-INFO
writing dependency_links to fastica.egg-info/dependency_links.txt
writing top-level names to fastica.egg-info/top_level.txt
reading manifest file 'fastica.egg-info/SOURCES.txt'
writing manifest file 'fastica.egg-info/SOURCES.txt'
running check
creating fastica-1.0
creating fastica-1.0/fastica.egg-info
copying files to fastica-1.0...
copying README.md -> fastica-1.0
copying fastica.py -> fastica-1.0
copying setup.py -> fastica-1.0
copying fastica.egg-info/PKG-INFO -> fastica-1.0/fastica.egg-info
copying fastica.egg-info/SOURCES.txt -> fastica-1.0/fastica.egg-info
copying fastica.egg-info/dependency_links.txt -> fastica-1.0/fastica.egg-info
copying fastica.egg-info/top_level.txt -> fastica-1.0/fastica.egg-info
Writing fastica-1.0/setup.cfg
Creating tar archive
removing 'fastica-1.0' (and everything under it)


In [29]:
%%file __init__.py
 
__author__ = 'zhechang'  
__all__=["fastica"]

Overwriting __init__.py


In [30]:
%%bash
python setup.py install

running install
running bdist_egg
running egg_info
writing fastica.egg-info/PKG-INFO
writing dependency_links to fastica.egg-info/dependency_links.txt
writing top-level names to fastica.egg-info/top_level.txt
reading manifest file 'fastica.egg-info/SOURCES.txt'
writing manifest file 'fastica.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
copying build/lib/fastica.py -> build/bdist.linux-x86_64/egg
byte-compiling build/bdist.linux-x86_64/egg/fastica.py to fastica.cpython-36.pyc
creating build/bdist.linux-x86_64/egg/EGG-INFO
installing scripts to build/bdist.linux-x86_64/egg/EGG-INFO/scripts
running install_scripts
running build_scripts
creating build/bdist.linux-x86_64/egg/EGG-INFO/scripts
copying build/scripts-3.6/fastica.py -> build/bdist.linux-x86_64/egg/EGG-INFO/scripts
changing mode of build/bdist.linux-x86_64/egg/EGG-INFO/scripts/fastica.py to 755
copying fastica.egg-info/PKG-

zip_safe flag not set; analyzing archive contents...
error: [Errno 13] Permission denied: '/opt/conda/lib/python3.6/site-packages/easy-install.pth'


In [3]:
import fastica

In [6]:
import numpy as np
A = np.random.random((4,4))

In [11]:
fastica.fastICA(A)

(array([[-7.22821050e-01, -9.55911702e-01,  7.19526022e-01,
         -1.47471528e-01],
        [ 1.85538698e-01, -1.03693703e+00, -1.27201796e+00,
         -3.94252727e-01],
        [-3.79673416e+00,  1.72666611e+00, -1.69580077e+00,
         -8.56800969e-01],
        [-1.38992875e+15, -1.08139099e+15, -1.43509351e+15,
          6.82027880e+15]]),
 array([[ 0.49293207, -0.77570231, -0.02923837,  0.39300004],
        [-0.39761904, -0.57739843, -0.26787908, -0.66087135],
        [ 0.11133491, -0.13092599,  0.92856501, -0.32898319],
        [ 0.76584699,  0.21853003, -0.25525072, -0.54824272]]),
 array([[ 1.86850743e+15, -3.14209394e+15, -1.56414115e+15,
         -2.60660434e+15],
        [ 1.86850743e+15, -3.14209394e+15, -1.56414115e+15,
         -2.60660434e+15],
        [ 1.86850743e+15, -3.14209394e+15, -1.56414115e+15,
         -2.60660434e+15],
        [ 1.86850743e+15, -3.14209394e+15, -1.56414115e+15,
         -2.60660434e+15]]))

In [35]:
! cd build

In [36]:
! ls

build  fastica.egg-info  fastica.ipynb	__init__.py  README.md
dist   fastICA.egg-info  fastica.py	__pycache__  setup.py
